In [1]:
import sys
import os

In [2]:
MODEL_DIR = os.path.realpath(os.path.dirname('..\superscript_model'))
sys.path.append(os.path.normpath(MODEL_DIR))

In [3]:
from superscript_model import model

In [4]:
model = model.SuperScriptModel(10)

In [5]:
model.run_model(100)